In [36]:
import jupyros as jp
import rospy
from final_assignment.srv import *
import ipywidgets as widgets
from IPython.display import display

import roslaunch

from final_assignment.srv import MoveBaseTarget, MoveBaseResult
from std_srvs.srv import *
from geometry_msgs.msg import Twist

from math import *
import time
import random

# List containing all the possible target location in the simulation
TARGET_POSE = [(-4, -3), (-4, 2), (-4, 7), (5, -7), (5, -3), (5, 1)]

target2 = MoveBaseTarget()
algo = 0
path_planner = ['Move Base: Dijkstra', 'Bug0']
task_open = False

In [37]:
class UserInteraction():
    def __init__(self):
        self.choice = ''
        self.value = value = {'None': 7,'1. Reach a (x,y) coordinate':1,'2. Drive the robot with the keyboard':2,'3. Drive the robot with some assistance':3,'0. Exit':0}
        self.out_2 = widgets.Output(layout={'border': '1px solid black'})
        
    def on_button_clicked(self, b):
        rospy.wait_for_service("/reachCoordinateService")
        client = rospy.ServiceProxy("/reachCoordinateService", reachCoordinateService)

        rt = client(self.x.value,self.y.value)


        with self.out_2:
            if rt.ret == 0:
                print("Target not reached, goal cancelled")
            elif rt.ret == 1:
                print("Target reached!")
        return self.out_2
    
    def control(self):
        self.out_1 = widgets.Output(layout={'border': '1px solid black'})

        widgets.Dropdown.value.tag(sync=True) 

        self.selection = widgets.Dropdown(
            options=['None','1. Reach a (x,y) coordinate','2. Drive the robot with the keyboard','3. Drive the robot with some assistance','0. Exit'],
            value= 'None',
            description='Control:'
        )

        self.display_choice = widgets.Text(value='None', description='You Selected:', disabled=True)

        mylink = widgets.jslink((self.selection, 'value'), (self.display_choice, 'value'))
        

        with self.out_1:
            print("Select Your Control Method:")
            display(self.selection, self.display_choice)
        return self.out_1
    
    def check_location(self, x, y):
        
        if (x, y) in TARGET_POSE:
            return True
        else:
            return False

    def start_task(self):
        rospy.loginfo("starting...")

        package ="teleop_twist_keyboard"

        executable ="teleop_twist_keyboard.py"
        node = roslaunch.core.Node(package, executable)

        launch = roslaunch.scriptapi.ROSLaunch()
        launch.start()
        task = launch.launch(node)
        print(task.is_alive())



    def call_movebase(self, target):
        
        rospy.wait_for_service('movebase_client')
        try:
            movebase_client = rospy.ServiceProxy('movebase_client', MoveBaseTarget)
            res = movebase_client(target.cord_x, target.cord_y)
            return res.status
        except rospy.ServiceException as e:
            print(f'Service call failed: {e}')


    def call_bug_algo(self, target):
       
        # sends the x and y coordinate to a parameter server
        rospy.set_param("des_pos_x", target.cord_x)
        rospy.set_param("des_pos_y", target.cord_y)
        print("Thanks! Let's reach the next position")
        rospy.wait_for_service('bug_switch')
        try:
            bug0 = rospy.ServiceProxy('bug_switch', MoveBaseResult)
            res = bug0('Are you there yet!')
            return res.status
        except rospy.ServiceException as e:
            print(f'Service call failed: {e}')


    def wait_for_result(self):
        
        rospy.wait_for_service('movebase_result')
        try:
            movebase_result = rospy.ServiceProxy('movebase_result', MoveBaseResult)
            res = movebase_result('Are you there yet!')
            return res.status
        except rospy.ServiceException as e:
            print(f'Service call failed: {e}')
    

    def execute(self, pick):
        vel_pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
        global algo
        state = 1
        # pick = 0
        global task_open
        global TARGET_POSE

        if (pick == 1 and state == 1):

            target = self.call_bug_algo(random.choice(TARGET_POSE))
            if (algo == 0):
                resp = self.call_movebase(target)
                while (resp):
                    target = self.call_bug_algo()
                    resp = self.call_movebase(target)
                wait_for_result()
                state = 0
            else:
                resp = self.call_bug_algo(target2)
                print(resp)
                algo = 0
                state = 0

        elif (pick == 2 and state == 1):
            self.x = widgets.IntText(value= 0, description='Select X Coordinate', disabled=False)
            self.y = widgets.IntText(value= 0, description='Select Y Coordinate', disabled=False)
            self.ready_button = widgets.Button(description='Lets go',disabled=False,button_style='info')


            with self.out_2:
                print("You need to select the respective x and y coordinates")
                display(self.x,self.y, self.ready_button)
            self.out_2
            self.out_2.clear_output(wait=True)
            self.ready_button.on_click(self.on_button_clicked)
            
            if (self.check_location(self.x, self.y)):
                target2.cord_x = self.x
                target2.cord_y = self.y
                if (algo == 0):
                    resp = self.call_movebase(target2)
                    if(resp):
                        print('The Robot is already at this location')
                    else:
                        self.wait_for_result()
                        state = 0
                else:
                    resp = self.call_bug_algo(target2)
                    print(resp)
                    algo = 0
                    state = 0
            else:
                print('Please enter one of the possible positions')
        elif (pick == 3 and state == 1 and task_open==False):
            print("Opening Teleop")
            self.start_task()
            task_open = True

        elif (pick not in range(1, 3)):
            state = 0

        
    def runros(self):
        rospy.init_node('userInteractionNode')

        if self.value[self.display_choice.value] == 1:
            return self.execute(1)
        elif self.value[self.display_choice.value] == 2:
            return self.execute(2)
        elif self.value[self.display_choice.value] == 3:
            return self.execute(3)
        else:
            with self.out_2:
                print("Make A Valid Selection")
            return self.out_2
        return self.out_2


## UI = UserInteraction()
UI.control()

In [38]:
UI.runros()

AttributeError: 'tuple' object has no attribute 'cord_x'